##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Detección de objetos con Model Maker de TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/object_detection"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/models/modify/model_maker/object_detection.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

En este bloc de colab, aprenderá a usar la librería [Model Maker de TensorFlow Lite](https://www.tensorflow.org/lite/models/modify/model_maker) para entrenar un modelo personalizado de detección de objetos capaz de detectar ensaladas dentro de imágenes en un dispositivo móvil.

La librería Model Maker usa *aprendizaje por transferencia* para simplificar el proceso de entrenamiento de un modelo TensorFlow Lite usando un conjunto de datos personalizado. Volver a entrenar un modelo TensorFlow Lite con su propio conjunto de datos personalizado reduce la cantidad de datos de entrenamiento necesarios y acortará el tiempo de entrenamiento.

Usará el conjunto de datos *Salads*, de acceso público, creado a partir del [Conjunto de datos de imágenes abiertas V4](https://storage.googleapis.com/openimages/web/index.html).

Cada imagen del conjunto de datos contiene objetos etiquetados como una de las clases siguientes:

- Productos horneados
- Queso
- Ensalada
- Mariscos
- Tomate

El conjunto de datos contiene los recuadros delimitadores que especifican dónde se sitúa cada objeto, junto con la etiqueta del objeto.

Aquí tiene una imagen de ejemplo del conjunto de datos:

<br>

<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" width="400" hspace="0">


## Requisitos previos


### Instalar los paquetes necesarios

Comience instalando los paquetes necesarios, incluido el paquete Model Maker del repositorio [GitHub](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) y la librería pycocotools que usará para la evaluación.

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

Importe los paquetes necesarios.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Preparar el conjunto de datos

Aquí usará el mismo conjunto de datos que el [inicio rápido](https://cloud.google.com/vision/automl/object-detection/docs/edge-quickstart#preparing_a_dataset) de AutoML.

El conjunto de datos *Salads* está disponible en: `gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv`.

Contiene 175 imágenes para el entrenamiento, 25 imágenes para la validación y 25 imágenes para las pruebas. El conjunto de datos tiene cinco clases: `Salad`, `Seafood`, `Tomato`, `Baked goods` y `Cheese`.

<br>

El conjunto de datos se entrega en formato CSV:

```
TRAINING,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Salad,0.0,0.0954,,,0.977,0.957,,
VALIDATION,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Seafood,0.0154,0.1538,,,1.0,0.802,,
TEST,gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg,Tomato,0.0,0.655,,,0.231,0.839,,
```

- Cada fila corresponde a un objeto localizado dentro de una imagen mayor, y cada objeto se designa específicamente como datos de prueba, entrenamiento o validación. Aprenderá más sobre lo que esto significa en una fase posterior de este bloc.
- Las tres líneas incluidas aquí indican **tres objetos distintos situados dentro de la misma imagen** disponible en `gs://cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg`.
- Cada fila tiene una etiqueta diferente: `Salad`, `Seafood`, `Tomato`, etc.
- Las cajas delimitadoras se especifican para cada imagen usando los vértices superior izquierdo e inferior derecho.

Esta es una visualización de estas tres líneas:

<br>

<img src="https://cloud.google.com/vision/automl/object-detection/docs/images/quickstart-preparing_a_dataset.png" width="400" hspace="100">

Si desea saber más sobre cómo preparar su propio archivo CSV y los requisitos mínimos para crear un conjunto de datos válido, consulte la guía [Preparación de sus datos de entrenamiento](https://cloud.google.com/vision/automl/object-detection/docs/prepare) para saber más.

Si es nuevo en Google Cloud, puede que se pregunte qué significa la URL `gs://`. Son URL de archivos almacenados en [Google Cloud Storage](https://cloud.google.com/storage) (GCS). Si hace públicos sus archivos en GCS o [autentica su cliente](https://cloud.google.com/storage/docs/authentication#libauth), Model Maker puede leer esos archivos de forma similar a sus archivos locales.

Sin embargo, no necesita mantener sus imágenes en Google Cloud para usar Model Maker. Puede usar una ruta local en su archivo CSV y Model Maker funcionará sin problema.

## Inicio rápido

El entrenamiento de un modelo de detección de objetos consta de seis pasos:

**Paso 1. Seleccione una arquitectura de modelo de detección de objetos.**

Este tutorial usa el modelo EfficientDet-Lite0. EfficientDet-Lite[0-4] son una familia de modelos de detección de objetos aptos para móviles/IoT derivados de la arquitectura [EfficientDet](https://arxiv.org/abs/1911.09070).

Este es el rendimiento de cada uno de los modelos EfficientDet-Lite en comparación con los demás.

Arquitectura modelo | Tamaño (MB)* | Latencia (ms)** | Precisión promedio***
--- | --- | --- | ---
EfficientDet-Lite0 | 4.4 | 37 | 25.69 %
EfficientDet-Lite1 | 5.8 | 49 | 30.55 %
EfficientDet-Lite2 | 7.2 | 69 | 33.97 %
EfficientDet-Lite3 | 11.4 | 116 | 37.70 %
EfficientDet-Lite4 | 19.9 | 260 | 41.96 %

<i> * Tamaño de los modelos cuantizados enteros. <br> ** Latencia medida en Pixel 4 usando 4 hilos en la CPU. <br> *** La precisión promedio es la mAP (precisión promedio media) en el conjunto de datos de validación COCO 2017.</i>


In [ ]:
spec = model_spec.get('efficientdet_lite0')

**Paso 2. Cargue el conjunto de datos.**

Model Maker tomará los datos de entrada en formato CSV. Usará el método `object_detector.DataLoader.from_csv` para cargar el conjunto de datos y dividirlos en las imágenes de entrenamiento, validación y prueba.

- Imágenes de entrenamiento: Estas imágenes se usan para entrenar el modelo de detección de objetos para reconocer los ingredientes de la ensalada.
- Imágenes de validación: Son imágenes que el modelo no vio durante el proceso de entrenamiento. Las usará para decidir cuándo debe detener el entrenamiento, para evitar [un ajuste excesivo](https://en.wikipedia.org/wiki/Overfitting).
- Imágenes de prueba: Estas imágenes se usan para evaluar el rendimiento final del modelo.

Puede cargar el archivo CSV directamente desde Google Cloud Storage, pero no necesita mantener sus imágenes en Google Cloud para usar Model Maker. Puede indicar un archivo CSV local en su computadora y Model Maker funcionará perfectamente.

In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

**Paso 3. Entrene el modelo TensorFlow con los datos de entrenamiento.**

- El modelo EfficientDet-Lite0 usa `epochs = 50` de manera predeterminada, lo que significa que recorrerá el conjunto de datos de entrenamiento 50 veces. Puede observar la precisión de validación durante el entrenamiento y detenerlo antes para evitar un ajuste excesivo.
- Configure `batch_size = 8` aquí para que vea que se necesitan 21 pasos para recorrer las 175 imágenes del conjunto de datos de entrenamiento.
- Configure `train_whole_model=True` para ajustar todo el modelo en lugar de entrenar sólo la capa de la cabeza para mejorar la precisión. Por otro lado, puede llevar más tiempo entrenar el modelo.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

**Paso 4. Evalúe el modelo con los datos de prueba.**

Tras entrenar el modelo de detección de objetos usando las imágenes del conjunto de datos de entrenamiento, use las 25 imágenes restantes del conjunto de datos de prueba para evaluar cómo se comporta el modelo frente a nuevos datos que nunca ha visto antes.

Como el tamaño predeterminado del lote es 64, se necesitará 1 paso para recorrer las 25 imágenes del conjunto de datos de prueba.

Las métricas de evaluación son las mismas que [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(test_data)

**Paso 5. Exporte como modelo TensorFlow Lite.**

Exporte el modelo entrenado de detección de objetos al formato TensorFlow Lite especificando a qué carpeta desea exportar el modelo cuantizado. La técnica de cuantización predeterminada tras el entrenamiento es la cuantización entera completa.

In [ ]:
model.export(export_dir='.')

**Paso 6. Evalúe el modelo TensorFlow Lite.**

Varios factores pueden afectar a la precisión del modelo al exportarlo a TFLite:

- La [cuantización](https://www.tensorflow.org/lite/performance/model_optimization) ayuda a reducir 4 veces el tamaño del modelo a costa de una cierta disminución de la precisión.
- El modelo TensorFlow original usa [supresión no máxima (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) por clase para el posprocesamiento, mientras que el modelo TFLite usa NMS global que es mucho más rápido pero menos preciso. Keras produce un máximo de 100 detecciones, mientras que tflite produce un máximo de 25 detecciones.

Por lo tanto, tendrá que evaluar el modelo TFLite exportado y comparar su precisión con la del modelo TensorFlow original.

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

Puede descargar el archivo del modelo TensorFlow Lite usando la barra lateral izquierda de Colab. Haga clic con el botón derecho en el archivo `model.tflite` y seleccione `Download` para descargarlo en su computadora local.

Este modelo puede integrarse en una app Android o iOS usando la [API ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) de la [librería de tareas TensorFlow Lite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

Consulte la [App de ejemplo de detección de objetos TFLite](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android) para saber más sobre cómo se usa el modelo en una app en funcionamiento.

*Nota: La vinculación de modelos de Android Studio aún no admite la detección de objetos, por lo que deberá usar la librería de tareas de TensorFlow Lite.*

## (Opcional) Pruebe el modelo TFLite en su imagen

Puede probar el modelo TFLite entrenado usando imágenes de Internet.

- Reemplace la `INPUT_IMAGE_URL` siguiente por la imagen de entrada que desee.
- Modifique el `DETECTION_THRESHOLD` para cambiar la sensibilidad del modelo. Un umbral más bajo significa que el modelo captará más objetos, pero también habrá más falsas detecciones. Mientras tanto, un umbral más alto significa que el modelo sólo captará objetos que haya detectado con seguridad.

Aunque en este momento se requiere algo de código repetitivo para ejecutar el modelo en Python, la integración del modelo en una app móvil sólo requiere unas pocas líneas de código.

In [ ]:
#@title Load the trained TFLite model and define some visualization functions

import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image_path,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
#@title Run object detection and show the detection results

INPUT_IMAGE_URL = "https://storage.googleapis.com/cloud-ml-data/img/openimage/3/2520/3916261642_0a504acd60_o.jpg" #@param {type:"string"}
DETECTION_THRESHOLD = 0.3 #@param {type:"number"}

TEMP_FILE = '/tmp/image.png'

!wget -q -O $TEMP_FILE $INPUT_IMAGE_URL
im = Image.open(TEMP_FILE)
im.thumbnail((512, 512), Image.ANTIALIAS)
im.save(TEMP_FILE, 'PNG')

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    TEMP_FILE,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
Image.fromarray(detection_result_image)

## (Opcional) Compilar para la TPU Edge

Ahora que dispone de un modelo EfficientDet Lite cuantizado, es posible compilarlo e implementarlo en una [Coral EdgeTPU](https://coral.ai/).

**Paso 1. Instale el compilador EdgeTPU**

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

**Paso 2. Seleccione el número de Edge TPUs, Compile**

La EdgeTPU dispone de 8MB de SRAM para almacenar en caché los parámetros del modelo ([más información](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching)). Esto significa que para los modelos de más de 8MB, el tiempo de inferencia aumentará para transferir los parámetros del modelo. Una forma de evitar esto es [Model Pipelining](https://coral.ai/docs/edgetpu/pipeline/) - dividir el modelo en segmentos que puedan tener una EdgeTPU dedicada. Esto puede mejorar significativamente la latencia.

La tabla siguiente se puede utilizar como referencia para el número de Edge TPU que se deben usar; los modelos más grandes no se compilarán para una sola TPU ya que los tensores intermedios no caben en la memoria en chip.

Arquitectura modelo | TPUs mínimos | TPUs recomendados
--- | --- | ---
EfficientDet-Lite0 | 1 | 1
EfficientDet-Lite1 | 1 | 1
EfficientDet-Lite2 | 1 | 2
EfficientDet-Lite3 | 2 | 2
EfficientDet-Lite4 | 2 | 3

In [ ]:
NUMBER_OF_TPUS =  1#@param {type:"number"}

!edgetpu_compiler model.tflite --num_segments=$NUMBER_OF_TPUS

**Paso 3. Descargar, ejecutar el modelo**

Con el modelo o modelos compilados, ya pueden ejecutarse en EdgeTPU(s) para la detección de objetos. En primer lugar, descargue el archivo del modelo TensorFlow Lite compilado usando la barra lateral izquierda de Colab. Haga clic con el botón derecho en el archivo `model_edgetpu.tflite` y seleccione `Download` para descargarlo en su computadora local.

Ahora puede ejecutar el modelo de la forma que prefiera. Algunos ejemplos de detección son:

- [detección pycoral](https://github.com/google-coral/pycoral/blob/master/examples/detect_image.py)
- [detección básica de TFLite](https://github.com/google-coral/tflite/tree/master/python/examples/detection)
- [detección de ejemplo de vídeo](https://github.com/google-coral/examples-camera)
- [API libcoral en C++](https://github.com/google-coral/libcoral)


## Uso avanzado

Esta sección cubre temas de uso avanzado como el ajuste del modelo y los hiperparámetros de entrenamiento.

### Cargar el conjunto de datos

#### Cargue sus propios datos

Puede subir su propio conjunto de datos para trabajar con este tutorial. Suba su conjunto de datos usando la barra lateral izquierda en Colab.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_object_detection.png" width="1000" hspace="0" alt="Subir archivo">

Si prefiere no subir su conjunto de datos a la nube, también puede ejecutar localmente la librería siguiendo la [guía](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

#### Cargue sus datos con un formato de datos diferente.

La librería Model Maker también admite el método `object_detector.DataLoader.from_pascal_voc` para cargar datos con formato [PASCAL VOC](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5#:~:text=Pascal%20VOC%20is%20an%20XML,for%20training%2C%20testing%20and%20validation). [makesense.ai](https://www.makesense.ai/) y [LabelImg](https://github.com/tzutalin/labelImg) son las herramientas que pueden anotar la imagen y guardar las anotaciones como archivos XML en formato de datos PASCAL VOC:

```python
object_detector.DataLoader.from_pascal_voc(image_dir, annotations_dir, label_map={1: "person", 2: "notperson"})
```


### Personalice los hiperparámetros del modelo EfficientDet

Los parámetros del modelo y del canal de entrenamiento que puede ajustar son:

- `model_dir`: La ubicación para guardar los archivos de puntos de verificación del modelo. Si no está activa, se usará un directorio temporal.
- `steps_per_execution`: Número de pasos por ejecución del entrenamiento.
- `moving_average_decay`: Float. El decaimiento a usar para mantener los promedios móviles de los parámetros entrenados.
- `var_freeze_expr`: La expresión regular para mapear el nombre del prefijo de las variables a congelar que significa permanecer igual durante el entrenamiento. Más concretamente, use `re.match(var_freeze_expr, variable_name)` en la base de código para mapear las variables a congelar.
- `tflite_max_detections`: entero, 25 de forma predeterminada. El número máximo de detecciones de salida en el modelo TFLite.
- `strategy`:  Una cadena que especifica qué estrategia de distribución usar. Los valores aceptados son 'tpu', 'gpus', Ninguno. 'tpu' significa usar TPUStrategy. 'gpus' significa usar MirroredStrategy para multi-gpus. Si es Ninguno, usa TF predeterminado con OneDeviceStrategy.
- `tpu`:  La TPU en la nube que se usará para el entrenamiento. Debe ser el nombre usado al crear la TPU en la nube o una url grpc://ip.direccion.de.la.tpu:8470.
- `use_xla`: Use XLA aunque la estrategia no sea tpu. Si la estrategia es tpu, siempre se usa XLA, y este Indicador no tiene efecto.
- `profile`: Habilitar el modo de perfil.
- `debug`: Habilite el modo de depuración.

Otros parámetros que pueden ajustarse se muestran en [hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170).

Por ejemplo, puede configurar el `var_freeze_expr='efficientnet'` que congela las variables con prefijo de nombre `efficientnet` (de forma<br>predeterminada es `'(efficientnet|fpn_cells|resample_p6)'`). Esto permite que el modelo congele las variables no entrenables y mantenga su valor igual a lo largo del entrenamiento.

```python
spec = model_spec.get('efficientdet_lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### Cambiar la arquitectura del modelo

Puede cambiar la arquitectura del modelo cambiando el `model_spec`. Por ejemplo, cambie el `model_spec` por el modelo EfficientDet-Lite4.

```python
spec = model_spec.get('efficientdet_lite4')
```

### Afinar los hiperparámetros de entrenamiento

La función `create` es la función maestra que usa la librería Model Maker para crear modelos. El parámetro `model_spec` define la especificación del modelo. Actualmente se admite la clase `object_detector.EfficientDetSpec`. La función `create` consta de los siguientes pasos:

1. Crea el modelo para la detección de objetos según `model_spec`.
2. Entrena el modelo. Las épocas predeterminadas y el tamaño del lote predeterminado se definen mediante las variables `epochs` y `batch_size` del objeto `model_spec`. También puede ajustar los hiperparámetros de entrenamiento como `epochs` y `batch_size` que afectan a la precisión del modelo. Por ejemplo:

- `epochs`: Entero, 50 de forma predeterminada. Más épocas podrían lograr una mayor precisión, pero pueden llevar a un sobreajuste.
- `batch_size`: Entero, 64 de forma predeterminada. El número de muestras a usar en un paso de entrenamiento.
- `train_whole_model`: Booleano, False de forma predeterminada. Si es true, entrene todo el modelo. En caso contrario, entrene sólo las capas que no coincidan con `var_freeze_expr`.

Por ejemplo, puede entrenar con menos épocas y sólo la capa de cabecera. Puede aumentar el número de épocas para obtener mejores resultados.

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

### Exportar a diferentes formatos

Los formatos de exportación pueden ser uno o una lista de los siguientes:

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`


De forma predeterminada, sólo exporta el archivo del modelo TensorFlow Lite que contiene los [metadatos](https://www.tensorflow.org/lite/models/convert/metadata) del modelo para que pueda usarlo posteriormente en una aplicación ML en el dispositivo. El archivo de etiquetas está incrustado en los metadatos.

En muchas aplicaciones de ML dentro de dispositivos, el tamaño del modelo es un factor importante. Por lo tanto, se recomienda cuantizar el modelo para hacerlo más pequeño y, potencialmente, ejecutarlo más rápido. En cuanto a los modelos EfficientDet-Lite, se usa la cuantización entera completa para cuantizar el modelo de forma predeterminada. Consulte [Cuantificación posterior al entrenamiento](https://www.tensorflow.org/lite/performance/post_training_quantization) para obtener más detalles.

```python
model.export(export_dir='.')
```

También puede seleccionar exportar otros archivos relacionados con el modelo para examinarlo mejor. Por ejemplo, exportando tanto el modelo guardado como el archivo de etiquetas de la siguiente manera:

```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### Personalice la cuantificación posterior al entrenamiento en el modelo TensorFlow Lite

[La cuantización posterior al entrenamiento](https://www.tensorflow.org/lite/performance/post_training_quantization) es una técnica de conversión que puede reducir el tamaño del modelo y la latencia de la inferencia, al tiempo que mejora la velocidad de inferencia de la CPU y del acelerador de hardware, con una pequeña degradación de la precisión del modelo. Por ello, se usa mucho para optimizar el modelo.

La librería Model Maker aplica una técnica predeterminada de cuantización postentrenamiento al exportar el modelo. Si desea personalizar la cuantización postentrenamiento, Model Maker también soporta múltiples opciones de cuantización postentrenamiento utilizando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Tomemos como ejemplo la cuantización float16. En primer lugar, defina la configuración de cuantificación.

```python
config = QuantizationConfig.for_float16()
```

A continuación, exportamos el modelo TensorFlow Lite con dicha configuración.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Más información

Puede leer nuestro [ejemplo de detección de objetos](https://www.tensorflow.org/lite/examples/object_detection/overview) para conocer los detalles técnicos. Para más información, consulte:

- [Guía](https://www.tensorflow.org/lite/models/modify/model_maker) y [Referencia de API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) de Model Maker de TensorFlow Lite.
- Librería de tareas: [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) para su implementación.
- Las apps de referencia de principio a fin: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios), y [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi).
